In [1]:
import pandas as pd

In [2]:
analysis_df = pd.read_csv('english_df.csv')
analysis_df


,title,artist,winner,year,lyrics,preprocessed_lyrics,language,anger_pct,anticipation_pct,disgust_pct,fear_pct,joy_pct,negative_pct,positive_pct,sadness_pct,surprise_pct,trust_pct,gloom_index
0,If I Ever Lose My Faith In You (Single),Sting,0,1993,Sorrow (syrup rmx) LyricsYea\nUh huh\nAbstract...,sorrow ( syrup rmx ) lyricsyea Uh huh abstract...,en,0.047511,0.067873,0.052036,0.095023,0.049774,0.117647,0.088235,0.076923,0.033937,0.063348,100.000000
1,Bad Guy,Billie Eilish,1,2019,TranslationsHrvatskiTürkçeSvenskaEspañolРусски...,translationshrvatskitürkçesvenskaespañolрусски...,en,0.074074,0.027778,0.074074,0.074074,0.023148,0.087963,0.023148,0.087963,0.013889,0.023148,87.623281
2,Help Me (Single),Joni Mitchell,0,1974,Lilith Lyrics[Verse 1]\nSeptember\nDont rain o...,lilith lyric [ vers 1 ] septemb dont rain dont...,en,0.018750,0.050000,0.025000,0.081250,0.037500,0.100000,0.050000,0.081250,0.012500,0.043750,84.932747
3,Not Ready To Make Nice,Dixie Chicks,1,2006,Not Ready to Make Nice Lyrics[Verse 1]\nForgiv...,"readi make nice lyric [ vers 1 ] forgiv , soun...",en,0.054852,0.054852,0.054852,0.063291,0.037975,0.101266,0.042194,0.084388,0.012658,0.042194,84.700788
4,Witchcraft (Single),Frank Sinatra,0,1958,Sweetest Maleficia Lyrics[Gilles de Rais:]\nI ...,sweetest maleficia lyric [ gill de rai : ] con...,en,0.047138,0.050505,0.060606,0.080808,0.037037,0.097643,0.067340,0.063973,0.016835,0.047138,83.425519
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
327,A Taste Of Honey,Herb Alpert And The Tijuana Brass,1,1965,A Taste of Honey Lyrics[Intro]\nA taste of hon...,tast honey lyric [ intro ] tast honey ! tast m...,en,0.000001,0.030612,0.000001,0.000001,0.020408,0.000001,0.112245,0.000001,0.020408,0.000001,4.155500
328,Sailing,Christopher Cross,1,1980,"Sailing Lyrics[Verse 1]\nWell, it's not far do...","sail lyric [ vers 1 ] well , 's far paradis le...",en,0.000001,0.020270,0.000001,0.000001,0.013514,0.027027,0.033784,0.000001,0.000001,0.006757,3.852200
329,If You Leave Me Now (Single),Chicago,0,1976,TranslationsTürkçeIf You Leave Me Now Lyrics[I...,translationstürkçeif leav lyric [ intro : char...,en,0.003356,0.003356,0.003356,0.003356,0.000001,0.003356,0.006711,0.003356,0.003356,0.003356,2.761490
330,She Drives Me Crazy (Single),Fine Young Cannibals,0,1989,She Drives Me Crazy (single remix) LyricsI can...,drive crazi ( singl remix ) lyricsi ca n't sto...,en,0.000001,0.006494,0.000001,0.000001,0.006494,0.006494,0.019481,0.000001,0.000001,0.012987,1.095636
